In [1]:
import pandas as pd
import numpy as np

In [2]:
file = open('anonymized-curriculum-access.txt').readlines()

In [3]:
## Remove Newline chars
for index,line in enumerate(file):
    file[index] = line.replace('\n','')

In [4]:
## Check length of first Entery
len(file[0].split())

6

In [5]:
## find which entries don't have the same length as first entry
non_standard_size = []
for index, line in enumerate(file):
    if len(line.split()) != 6:
        non_standard_size.append(f'index {index} is {len(line.split())} long')
                                 

In [6]:
print(non_standard_size[:15])

['index 411 is 5 long', 'index 412 is 5 long', 'index 425 is 5 long', 'index 435 is 5 long', 'index 436 is 5 long', 'index 437 is 5 long', 'index 438 is 5 long', 'index 439 is 5 long', 'index 440 is 5 long', 'index 441 is 5 long', 'index 442 is 5 long', 'index 443 is 5 long', 'index 444 is 5 long', 'index 445 is 5 long', 'index 446 is 5 long']



Check what a len5 entry looks like vs a len6 entry

In [7]:
file[411]

'2018-01-26 16:46:16 / 48  97.105.19.61'

In [8]:
file[0]

'2018-01-26 09:55:03 / 1 8 97.105.19.61'

Looks like len5 and len6 differ by 1 type of ID, so let's split then into two pieces 

In [9]:
logs= []
for index,line in enumerate(file):
    if len(line.split()) == 6:
        ## This mean it has all 6 fields and thus is student
        x = line.split()
        logs.append({'date_time': x[0]+' ' +x[1],'dir':x[2],'id':x[3],'cohort':x[4],'ip':x[5]})
    elif len(line.split()) == 5:
        x = line.split()
        logs.append({'date_time': x[0]+' ' +x[1],'dir':x[2],'id':x[3],'ip':x[4]})

In [10]:
print(f'formatted for dataframe logs length:{len(logs)} how many lines did we start with:{len(file)} ')

formatted for dataframe logs length:233528 how many lines did we start with:233528 


Both numbers are the same so we didn't lose any Lines, which means our two if statements covered all possibilities

In [41]:
df = pd.DataFrame(logs)
df.set_index('date_time',inplace=True)
df.index = pd.to_datetime(df.index)

In [42]:
df.cohort.value_counts(dropna=False)

24     34485
22     28268
23     26655
25     24681
26     24088
29     20471
27     19255
NaN    14322
1       8821
14      6523
21      6465
28      6163
31      2997
13      2163
17      1805
8       1671
18      1482
19       901
16       740
15       632
7        338
12       230
nan      136
2         89
11        79
6         59
9          5
4          4
Name: cohort, dtype: int64

So we have some as NaN and some as nan. Let's switch those values that are nan to NaN

In [43]:
df['cohort'].replace('nan',np.nan,inplace=True)

In [44]:
students = df[df['cohort'].notnull()]
staff = df[df['cohort'].isnull()]

In [45]:
cohort_22 = students[students['cohort']=='22'].drop(columns='cohort')

In [56]:
cohort_22['day_of_week'] = cohort_22.index.weekday

In [60]:
cohort_22['weekend'] = cohort_22['day_of_week'] > 4

These are the Value counts of the IP addresses for Cohort 22 given that it's not a weekend

In [82]:
c22_ip_given_weekday = pd.DataFrame(cohort_22[cohort_22['weekend'] == False].ip.value_counts())

In [98]:
c22_ip_given_weekday['%chance of IP'] = (c22_ip_given_weekday['ip']/c22_ip_given_weekday.ip.sum())*100

In [119]:
c22_ip_given_weekday['outlier'] = c22_ip_given_weekday['%chance of IP'] < 0.01

In [121]:
c22_ip_given_weekday[c22_ip_given_weekday.outlier == True]

,ip,%chance of IP,outlier
99.203.26.185,2,0.007913,True
90.85.119.75,2,0.007913,True
166.251.112.88,2,0.007913,True
107.77.80.82,2,0.007913,True
98.6.222.42,2,0.007913,True
107.77.80.57,2,0.007913,True
107.77.100.45,2,0.007913,True
107.77.72.112,2,0.007913,True
99.203.26.200,2,0.007913,True
107.77.66.96,2,0.007913,True
